In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime as dt

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/online-retail-ii-uci/online_retail_II.csv


###############################################################
# Business Problem
###############################################################

* An e-commerce company wants to segment its customers and determine marketing strategies according to these segments.

###############################################################
# Data Understanding
###############################################################

Attribute Information:

* InvoiceNo: Invoice number. Nominal. A 6-digit integral number uniquely assigned to each transaction. If this code starts with the letter 'c', it indicates a cancellation.
* StockCode: Product (item) code. Nominal. A 5-digit integral number uniquely assigned to each distinct product.
* Description: Product (item) name. Nominal.
* Quantity: The quantities of each product (item) per transaction. Numeric.
* InvoiceDate: Invice date and time. Numeric. The day and time when a transaction was generated.
* UnitPrice: Unit price. Numeric. Product price per unit in USD.
* CustomerID: Customer number. Nominal. A 5-digit integral number uniquely assigned to each customer.
* Country: Country name. Nominal. The name of the country where a customer resides.

In [2]:
df_ = pd.read_csv("../input/online-retail-ii-uci/online_retail_II.csv")
df = df_.copy() #this code protects df_ if we need it we can use it again.
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95000,13085.00000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10000,13085.00000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25000,13085.00000,United Kingdom


In [3]:
# Let's examine the general structural features
print("##################### Index #####################")
print(df.index)
print("##################### Shape #####################")
print(df.shape)
print("##################### Types #####################")
print(df.dtypes)
print("##################### NA #####################")
print(df.isnull().sum())
print("##################### Total NA #####################")
print(df.isnull().sum().sum())
print("##################### Describe #####################")
print(df.describe().T)

##################### Index #####################
RangeIndex(start=0, stop=1067371, step=1)
##################### Shape #####################
(1067371, 8)
##################### Types #####################
Invoice         object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
Price          float64
Customer ID    float64
Country         object
dtype: object
##################### NA #####################
Invoice             0
StockCode           0
Description      4382
Quantity            0
InvoiceDate         0
Price               0
Customer ID    243007
Country             0
dtype: int64
##################### Total NA #####################
247389
##################### Describe #####################
                    count        mean        std          min         25%  \
Quantity    1067371.00000     9.93890  172.70579 -80995.00000     1.00000   
Price       1067371.00000     4.64939  123.55306 -53594.36000     1.25000   
Customer ID  8243

In [4]:
# We do not need to struggle with the NA for this analysis so I have dropped them
df.dropna(inplace=True)

In [5]:
# If this code starts with the letter 'c', it indicates a cancellation. I took out them
df = df[~df["Invoice"].str.contains("C", na=False)] 

In [6]:
# We calculate the total price for monetary value
df["TotalPrice"] = df["Quantity"] * df["Price"]

In [7]:
today_date = dt.datetime(2011, 12, 11)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 805620 entries, 0 to 1067370
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Invoice      805620 non-null  object 
 1   StockCode    805620 non-null  object 
 2   Description  805620 non-null  object 
 3   Quantity     805620 non-null  int64  
 4   InvoiceDate  805620 non-null  object 
 5   Price        805620 non-null  float64
 6   Customer ID  805620 non-null  float64
 7   Country      805620 non-null  object 
 8   TotalPrice   805620 non-null  float64
dtypes: float64(3), int64(1), object(5)
memory usage: 61.5+ MB


In [9]:
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"]) 

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 805620 entries, 0 to 1067370
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      805620 non-null  object        
 1   StockCode    805620 non-null  object        
 2   Description  805620 non-null  object        
 3   Quantity     805620 non-null  int64         
 4   InvoiceDate  805620 non-null  datetime64[ns]
 5   Price        805620 non-null  float64       
 6   Customer ID  805620 non-null  float64       
 7   Country      805620 non-null  object        
 8   TotalPrice   805620 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 61.5+ MB


In [11]:
rfm = df.groupby("Customer ID").agg({"InvoiceDate": lambda x: (today_date - x.max()).days,
                                    "Invoice": lambda x: x.nunique(),
                                    "TotalPrice": lambda x: x.sum()})

In [12]:
rfm = rfm[rfm["TotalPrice"] > 0]

In [13]:
rfm.columns = ["Recency", "Frequency", "Monetary"] 

In [14]:
rfm["recency_score"] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1])

In [15]:
rfm["frequency_score"] = pd.qcut(rfm["Frequency"].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])

In [16]:

rfm["monetary_score"] = pd.qcut(rfm["Monetary"], 5, labels=[1, 2, 3, 4, 5])


In [17]:
rfm["RFM_SCORE"] = (rfm['recency_score'].astype(str) +
                    rfm['frequency_score'].astype(str) + rfm['monetary_score'].astype(str))

In [18]:
rfm.head()

,Recency,Frequency,Monetary,recency_score,frequency_score,monetary_score,RFM_SCORE
Customer ID,,,,,,,
12346.00000,326,12,77556.46000,2,5,5,255
12347.00000,3,8,5633.32000,5,4,5,545
12348.00000,76,5,2019.40000,3,4,4,344
12349.00000,19,4,4428.69000,5,3,5,535
12350.00000,311,1,334.40000,2,1,2,212


In [19]:
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

In [20]:
rfm['segment'] = (rfm['recency_score'].astype(str) +rfm['frequency_score'].astype(str)).replace(seg_map, regex=True)

In [21]:
rfm.head()

,Recency,Frequency,Monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
Customer ID,,,,,,,,
12346.00000,326,12,77556.46000,2,5,5,255,cant_loose
12347.00000,3,8,5633.32000,5,4,5,545,champions
12348.00000,76,5,2019.40000,3,4,4,344,loyal_customers
12349.00000,19,4,4428.69000,5,3,5,535,potential_loyalists
12350.00000,311,1,334.40000,2,1,2,212,hibernating


In [22]:
from IPython.display import display, HTML

display(HTML(rfm[["segment", "Recency", "Frequency", "Monetary"]].groupby("segment").agg(["mean", "median", "count"]).to_html()))

In [23]:
rfm[["segment", "Recency", "Frequency", "Monetary"]].groupby("segment").agg(["mean", "median", "count"])

Recency                 Frequency                 \
                         mean    median count      mean   median count   
segment                                                                  
about_to_sleep      107.61856  94.00000   388   1.36082  1.00000   388   
at_Risk             373.94267 377.00000   750   3.90400  4.00000   750   
cant_loose          331.80282 326.00000    71  15.92958 11.00000    71   
champions             9.25000   9.00000   852  19.21714 11.50000   852   
hibernating         460.01248 435.00000  1522   1.25230  1.00000  1522   
loyal_customers      68.30602  54.00000  1147   9.80384  8.00000  1147   
need_attention      114.24164 107.00000   269   3.15613  3.00000   269   
new_customers        11.30357  11.00000    56   1.00000  1.00000    56   
potential_loyalists  26.29734  24.00000   713   2.58906  3.00000   713   
promising            39.28182  38.50000   110   1.00000  1.00000   110   

                       Monetary                   
                           mean     median count  
segment                                           
about_to_sleep        531.96990  370.17500   388  
at_Risk              1383.59625  965.51000   750  
cant_loose           8355.67763 3881.89000    71  
champions           10795.52058 4009.87000   852  
hibernating           437.96314  285.20500  1522  
loyal_customers      4199.72859 2602.32000  1147  
need_attention       1283.11401  977.17000   269  
new_customers         356.25786  262.68000    56  
potential_loyalists  1155.42728  692.19000   713  
promising             324.49700  221.62000   110

In [24]:
rfm.groupby("segment").agg({'segment': 'count',
                            'Recency': ['mean', 'median', 'min', 'max'],
                            'Frequency': ['mean', 'median', 'min', 'max'],
                            'Monetary': ['mean', 'median', 'min', 'max']}).\
    sort_values(by=('segment', 'count'), ascending=False, axis=0)

segment   Recency                     Frequency           \
                      count      mean    median  min  max      mean   median   
segment                                                                        
hibernating            1522 460.01248 435.00000  191  739   1.25230  1.00000   
loyal_customers        1147  68.30602  54.00000   22  190   9.80384  8.00000   
champions               852   9.25000   9.00000    1   20  19.21714 11.50000   
at_Risk                 750 373.94267 377.00000  191  734   3.90400  4.00000   
potential_loyalists     713  26.29734  24.00000    1   59   2.58906  3.00000   
about_to_sleep          388 107.61856  94.00000   60  190   1.36082  1.00000   
need_attention          269 114.24164 107.00000   60  190   3.15613  3.00000   
promising               110  39.28182  38.50000   22   59   1.00000  1.00000   
cant_loose               71 331.80282 326.00000  193  577  15.92958 11.00000   
new_customers            56  11.30357  11.00000    1   20   1.00000  1.00000   

                                Monetary                                    
                    min  max        mean     median       min          max  
segment                                                                     
hibernating           1    2   437.96314  285.20500   2.95000  13916.34000  
loyal_customers       4  107  4199.72859 2602.32000  30.95000 144458.37000  
champions             4  398 10795.52058 4009.87000 357.00000 608821.65000  
at_Risk               2    8  1383.59625  965.51000  38.92000  44534.30000  
potential_loyalists   1    4  1155.42728  692.19000  20.80000 168472.50000  
about_to_sleep        1    2   531.96990  370.17500   6.20000   5416.32000  
need_attention        2    4  1283.11401  977.17000 120.90000  39916.50000  
promising             1    1   324.49700  221.62000  30.00000   1635.66000  
cant_loose            8  155  8355.67763 3881.89000 628.32000  77556.46000  
new_customers         1    1   356.25786  262.68000  43.20000   3861.00000